<h1>Tutorial: Mining Software Repositories</h1>

<h2>Introduction</h2>

<p>In this tutorial we will explore basic tasks of mining software repositoris and data collection procedures.</p> 

<p><strong>Mining Software Repositories (MSR) </strong>: field that analyzes source code repositories for the purpose of obtaining intersting information and actionable insigts about practical aspectes of software engineering. The data obtained from repositores is the bases of most empricial software engineering researches. </p>

<h3>Review of git version control system and PyDriller </h3>

<p> <a href="https://git-scm.com/">Git</a> is free and open soruce version control system. 
It helps to keep track of changes in files including source code, and 
helps devlopers to share and colaborate software projects. Git is avialable as GUI and command line tool. We use the command line tool for more flexibility</p>

<h4>Installation</h4>
<p>Checkout <a href="https://git-scm.com/downloads">this page</a> to get git for your system. You can check if git is already installed by typing <strong>git</strong> in the command line.</p>

In [ ]:
#1 check if git is already installed, if not installed download and 
!git

<p>We are going to explore some git commands by mining <a href="https://github.com/mtotschnig/MyExpenses">MyExpenses</a>
  project for the remaining exercises. This is an opensource android app for tracking expenses </p>

<h4>Cloning repository</h4>

In [ ]:
#clone the project MyExpenses, we are basically downloading the default branch of the source code.
!git clone https://github.com/mtotschnig/MyExpenses MyExpenses

In [ ]:
#Go to the project folder
%cd MyExpenses/
!pwd

In [ ]:
#check the status of the repository
!git status

In [ ]:
#check the branches of the repo 
!git branch
#!echo "-r option" #for also showing remote branches
#!git branch -r
#!echo "show-branch"
#!git show-branch #shows the default branch and the last commit message

<h4>Commits</h4>
<p>Commits are snapsots of a reposiory at different points in time. They are basic building blocks of repositories.</p>

In [ ]:
#use variations of git log command to explore commits associated with a repository
!git log #shows the commits from most recent to the oldest

In [ ]:
#!git log -n #view most recent n-commits
!git log -3

In [ ]:
!git log --oneline #just view single line per commit

In [ ]:
#git command to get the commit ids of list of commits associated with a branch, 
#helpful when scripting data collection. get rev-list <branch>
!git rev-list master

<h4>Checkingout a repsotory</h4>
<p>We use <strong>git checkout</strong> command to go to any snapshot of the repositoy in time using the commit id.
This is very important command when you need to analyzie (Eg. detect smells) every commit of a repository.</p>

In [ ]:
#First use git log or git rev-list to grab some commit id
!echo "before checkout"
!git status

In [ ]:
#then use checkout
!git checkout 1b4f4dc00dba7d41afc9d51641b196c38fbd7488

In [ ]:
!echo "after checkout"
!git status

In [ ]:
#to get back to the current state of the default branch <master> for this case
!git checkout master

In [ ]:
!git status #now its back to master

<h3>Using PyDriller</h3>
<p><a href="https://pydriller.readthedocs.io/en/latest/intro.html">PyDriller</a> is a python library/framework that helps devlopers and researchers to do MSR. It is specially helpful to extract information associated with commits and it implments szz algorithm to find bug inducing commits given bug fix commits.</p>

In [ ]:
#installation command
pip install pydriller

In [ ]:
#check if the installation is successful
from pydriller import Repository

<p><strong>Repository:</strong> is the main class of Pydriller to intialize a repository and traverse commits. To assign PyDriller a repository to analyze, you can use local path, remote url, list of local paths of repositories or urls. You can target specifc commits and commit ranges using data filters. You can analyze a single commit using <strong>single</strong> paramter by asssigning commit id. You can use <strong>since, from_commit, to, to_commit</strong> options to specifically analyzie a set of commits. The <strong>order </strong> parameter determines the order of the commit objects returned by this function. </p>

<p>Repository class have a function <strong>traverse_commits()</strong> This function retuns commit objects from the oldest commit to the newst commit (by default)</p>. 

In [ ]:
#Intialize pydriller with the cloned repository MyExpenses
#r=Repository("MyExpenses")
#r=Repository("MyExpenses",since=datetime.datetime(2021, 9, 1))

In [ ]:
#traversing commits with pydriller
for c in r.traverse_commits():
    print("Commit:{}\n time:{} \n".format(c.hash,c.committer_date))

<p> <strong>Commit</strong> object contains all informations associated wiht a git commit including hash, msg, author, committer, author_date,author_timezone, committer_date, modified_files(list of modifed files in the commit) and some statistics about the commit.</p>

<p><strong>ModifedFile</strong> object stores information about modifed file in a commit. It contians attributes such as old_path, new_path, filename, change_type,diff,diff_parsed, nloc, complexity,number_of_methods etc.</p>

In [ ]:
#PyDriller commit object properties
modified_files=[]
for commit in Repository("MyExpenses",single='52493a4a110fcb0fdbf292976bff129ae9a8ed26').traverse_commits():
    print("commit id: {}\n Commit message: {} \n Author name: {} \n Committer name: {} \n Committer date: {}".format(commit.hash, commit.msg,commit.author.name, commit.committer.name, commit.committer_date))
    modified_files=commit.modified_files

In [ ]:
#iterate through modifed files of the commit
for f in modified_files:
    print(f.filename)
    print(f.new_path)
    print(f.diff)
    print(f.diff_parsed) #very useful to extract lines of code where change happens

<h3>Selecting subject systems from GitHub</h3>

<p>The choice of subject systems for your study determines the quality of your research outcomes. 
Github repository is public and for anyone, so you could have archived projects, toy projects, class tutorials, introductory examples... Care should be taken to makesure the selected subject systems are approperate for your context. Please read the paper <a href="https://dl.acm.org/doi/10.1145/2597073.2597074"> The promises and perils of mining GitHub </a> to get more detailed information and on some techniques to make sure we get the right projects from GitHub.</p>

<ul>
    <li>Most projects have very few commits</li>
    <li>Most projects are inactive or active for very small time. Check this by checking the time of last commit of a project</li>
    <li>A large portion of repositories are not for software devlopment</li>
    <li>Two thirds of repostitoris are personal repositories</li>

</ul>

<p>When choosing projects, check the number of stars, number of forks, number of issues, number of open issues, relevance and most recent commit time to avoid toy projects.</p>

<h4>Typical subject system selection steps</h4>

<ol>
    <li>First, explore related literature if there are already popular subject systems and use or include them in your study. Eg. the paper considers projects in F-droid repositories. F-droid repsitores are popular in studies that involve android apps. If you don't find, you can go to the following steps</li>
    <li>Identify keywords that could describe your target projects</li>
    <li>Go to the github advanced search and apply the keywords. Check if the projects that you get are approperate. Rank them in decreasing order of stars to get more popular projects.</li>
    <li>If you are ok with the keywords, write a script to autmatically search repositores and gather your hits. There are many ways to do it, but <a href="https://docs.github.com/en/rest/reference/search">Github search API </a> and <a href="https://github.com/PyGithub/PyGithub">PyGitHub</a> library are popular for this activity.Watchout for ratelimits and other API constraints.
   </li>
    
</ol>

<p>There is no standard rule to decide the number of projects that are enough for study. But go for as higher as possible considering the avialable resources and data-collection time. Also, consider related work in this area to see what are the average number of subject systems used. It also depends on the type of study. Number of projects detrmines the generalizability of your results.

</p>

<p>
    <strong>Example:</strong> Lets use the <a href="https://github.com/search/advanced">Github advanced search</a> to look for machine learning projects in python
</p>

<h3>Code smell detection tools</h3>

<h4>Test smell detection using TsDetect</h4>

<ol>
    <li>First step is to detect test files, use <a href="https://github.com/TestSmells/TestFileDetector">TestFileDetector</a> </li>
    <li>Ts detect also needs to match test file to the production file, use <a href="https://github.com/TestSmells/TestFileMapping">Test File Mapping</a></li>
    <li>Prepare a CSV file containing test file name, production file name and run ts detect with the csv file as a parameter</li>
    <li>Repeat this steps for each version and for each subject systems</li>
    <li>Modify/adapt the (test file detect and test file mapping) script to work with python code. Use the <a href="https://github.com/TestSmells/PythonTestSmellDetector">python test smell detector.</a></li>
</ol>

<h4>Traditional code smell detection</h4>

<p>Demo: Use DECOR for traditional code smell detection in java</p>

In [ ]:
#!java -jar DECOR_JAVA.jar <project path> <project name>  ""

!java -jar DECOR_JAVA.jar MyExpenses MyExpenses  ""

<strong>The result of the detection is stored two folder lavels above the detector, make sure you consider that</strong>